In [64]:
import os
from langchain.document_loaders import PyPDFLoader
from typing import List, Tuple, Dict, Any
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.embeddings import DashScopeEmbeddings
from langchain.chains import RetrievalQA
from datasets import load_dataset
from langchain.schema import Document
import numpy as np
import json
import faiss
from langchain.chat_models import ChatOllama
from datasets import load_dataset
from evaluate import load
import time
import pandas as pd

# DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY")

In [65]:
from llama_cpp import Llama
from langchain.embeddings.base import Embeddings

# 自定义 LangChain 的 Embeddings 类封装
class LlamaCppEmbeddings(Embeddings):
    def __init__(self, model_path: str):
        self.llm = Llama(model_path=model_path, embedding=True)

    def embed_documents(self, texts: list[str]):
        # return [self.llm.embed(text)["data"][0]["embedding"] for text in texts]
        embeddings = []
        for text in texts:
            result = self.llm.embed(text)
            if isinstance(result, list) and isinstance(result[0], list):
                embeddings.append(result[0])
            else:
                embeddings.append(result)
        return embeddings

    def embed_query(self, text):
        # return self.llm.embed(text)["data"][0]["embedding"]
        result = self.llm.embed(text)
        return result[0] if isinstance(result, list) and isinstance(result[0], list) else result

In [66]:
class Proof():
    def __init__(self, document: Document, vector: List[np.ndarray], score: float):
        self.document = document
        self.vector = vector
        self.score = score
        

In [ ]:
class Client:
    """
    轻量级rag客户端，负责数据集加载、向量存储构建与检索。
    """
    def __init__(self, model_path: str = "./models/Qwen3-Embedding/Qwen3-Embedding-0.6B-Q8_0.gguf", 
                vectorstore_path: str = "faiss_db"): # dashscope_api_key: str,使用api调用embedding模型
        os.environ.setdefault("KMP_DUPLICATE_LIB_OK", "TRUE")
        self.vectorstore_path = vectorstore_path
        # self.embeddings = DashScopeEmbeddings(
        #     model="text-embedding-v1",
        #     dashscope_api_key=dashscope_api_key
        # )
        self.embeddings = LlamaCppEmbeddings(model_path=model_path)
        self.db: FAISS = None

    def _chunk_text(self, text: str, chunk_size=800, overlap= 200) -> list[str]:
        """
        将文本分块处理，使用递归字符分割器。
        """
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=overlap,
            separators=["\n\n", "\n", "。", ".", "！", "？", "!", "?", " ", ""],
            length_function=len
        )
        return splitter.split_text(text)

    # 读取PDF文件并提取文本内容
    def _read_pdfs(self, pdf_paths: List[str]) -> List[Document]:
        docs = []
        for path in pdf_paths:
            loader = PyPDFLoader(path)
            pages = loader.load_and_split()
            docs.extend(pages)
        return docs

    # 读取JSON文件夹中的所有文件
    def _load_json_folder(self, folder_path: str) -> List[Document]:
        docs = []
        for filename in os.listdir(folder_path):
            if not filename.endswith('.json'):
                continue
            filepath = os.path.join(folder_path, filename)
            with open(filepath, encoding='utf-8') as f:
                data = json.load(f)
            content = f"{data.get('title', '')}\n{data.get('content', '')}".strip()
            if content:
                docs.append(Document(page_content=content, metadata={'source': filepath}))
        return docs
    
    # 在线读取数据集
    def _streaming_load_dataset(self, sample_size=100, language='en', date_version='20231101') -> List[str]:
        # 启用streaming模式在线读取huggingface datasets
        dataset = load_dataset("wikimedia/wikipedia", f'{date_version}.{language}', streaming=True)
        docs = []
        for i, item in enumerate(dataset['train']):
            if i >= sample_size:
                break
            text = item.get('text', '')
            title = item.get('title', '')
            if not text:
                continue
            # snippet = text[:5000]
            meta = {'source': f'wikipedia://{language}/{item.get("id")}'}
            docs.append(Document(page_content=f"{title}\n{text}", metadata=meta))
        print(f"Streamed {len(docs)} Wikipedia docs.")
        return docs
    
    def build_vectorstore(self, sample_size=100, batch_size=10, 
                          streaming=False, folder_path=None, pdf_paths:List[str]=None):
        docs = []
        if streaming:
            # 在线读取数据集
            docs.extend(self._streaming_load_dataset(sample_size))
        elif folder_path is not None and pdf_paths is None:
            # 从指定文件夹加载JSON文件
            docs.extend(self._load_json_folder(folder_path))
        elif pdf_paths is not None:
            # 从PDF文件加载
            docs.extend(self._read_pdfs(pdf_paths))

        texts, metadatas = [], []
        faiss_id = 0
        # 分块并批量处理
        for i, doc in enumerate(docs):
            chunks = self._chunk_text(doc.page_content)
            for j, chunk in enumerate(chunks):
                texts.append(chunk)
                metadatas.append({
                "source": doc.metadata.get("source", ""),
                "doc_id": i,
                "chunk_id": j,
                "faiss_id": faiss_id
            })
                faiss_id += 1
                # 每 batch_size 保存一次，防止内存溢出
                if len(texts) >= batch_size or j == len(chunks) - 1:
                    if self.db is None:
                        self.db = FAISS.from_texts(texts, embedding=self.embeddings, metadatas=metadatas, normalize_L2=True)
                    else:
                        self.db.add_texts(texts, metadatas=metadatas)
                    texts.clear()
                    metadatas.clear()
            print(f"Processed {i+1}/{len(docs)} articles...")

        # 保存向量库
        if self.db:
            self.db.save_local(self.vectorstore_path)
            print(f"Vectorstore saved to {self.vectorstore_path}")
        else:
            print("No data processed.")

    def load_vectorstore(self) -> None:
        """
        加载已保存的向量存储
        """
        if not os.path.exists(self.vectorstore_path):
            raise FileNotFoundError(f"Vectorstore directory '{self.vectorstore_path}' not found.")
        self.db = FAISS.load_local(
            self.vectorstore_path,
            embeddings=self.embeddings,
            allow_dangerous_deserialization=True
        )
        print(f"Vectorstore {self.vectorstore_path} loaded.")

    def retrieve(self, query:str, top_k=4):
        """
        通过query在FAISS向量库中检索k个最相似文档，
        返回每个Document对象、其特征向量及相似度得分
        """
        # 检查向量库是否已加载
        if self.db is None:
            raise ValueError("Vectorstore尚未加载，请先调用load_vectorstore或build_vectorstore")

        # 查询向量并归一化
        query_vec = np.array(self.embeddings.embed_query(query), dtype=np.float32)
        query_norm = query_vec / np.linalg.norm(query_vec)

        # 使用 FAISS 内积搜索（等价于余弦相似度）
        scores, indices = self.db.index.search(query_norm.reshape(1, -1), top_k)

        results = []
        for i, idx in enumerate(indices[0]):
            if idx == -1:
                continue
            doc_id = self.db.index_to_docstore_id[idx]
            doc = self.db.docstore.search(doc_id)

            faiss_index = int(idx)
            vec = self.db.index.reconstruct(faiss_index).tolist()
            score = float(scores[0][i])  # 余弦相似度
            results.append(Proof(doc, vec, score))
        
        return results, query_vec

        # # 获取查询向量
        # query_vec = self.embeddings.embed_query(query)
        # query_vec = np.array(query_vec, dtype=np.float32)
        
        # # 使用FAISS索引进行搜索，获取L2距离和索引
        # distances, indices = self.db.index.search(np.array([query_vec], dtype=np.float32), top_k)

        # results = []
        # for i in range(top_k):
        #     index = indices[0][i]
        #     distance = distances[0][i]  # L2距离平方：||u - v||^2
        #     print(f"Index: {index}, Distance: {distance}")

        #     docstore_id = self.db.index_to_docstore_id[index]
        #     doc = self.db.docstore.search(docstore_id)

        #     faiss_index = int(index)
        #     vec = self.db.index.reconstruct(faiss_index)  # 重建文档向量
        #     vec = np.array(vec, dtype=np.float32)
        #     print(f"Document: {doc}, Vector: {vec}")

        #     # 计算向量范数
        #     query_norm = np.linalg.norm(query_vec)
        #     vec_norm = np.linalg.norm(vec)

        #     # 从L2距离计算内积：<u,v> = (||u||^2 + ||v||^2 - ||u - v||^2) / 2
        #     inner_product = (query_norm**2 + vec_norm**2 - distance) / 2

        #     # 计算余弦相似度
        #     cos_sim = inner_product / (query_norm * vec_norm) if query_norm * vec_norm != 0 else 0

        #     # 打包结果
        #     results.append(Proof(doc, vec.tolist(), cos_sim))
        # return results, query_vec

In [68]:
class Server:
    """
    Server 类，负责：
    1) 接收客户端选择的上下文数据
    2) 验证数据完整性（通过 Proof 信息）
    3) 调用 Ollama 部署的 Qwen3:4B 模型生成答案
    """
    def __init__(self, model_name: str = "qwen3:4b"):
        self.llm = ChatOllama(model=model_name)

    def verify_documents(self):
        return

    def build_prompt(self, query: str, contexts: List[str]) -> str:
        """构造 Prompt，将 query 和上下文拼接"""
        prompt = "You are an AI assistant. Use the following contexts to answer the question:\n"
        for i, c in enumerate(contexts, 1):
            prompt += f"Context {i}: {c}\n"
        prompt += f"Question: {query}\nAnswer:"
        return prompt

    def generate_answer(self, query: str, contexts: List[str]) -> str:
        """
        验证 Proof 后调用模型生成答案
        """
        # if not self.verify_documents(contexts, proofs):
        #     raise ValueError("Proof verification failed! Data may be tampered.")
        prompt = self.build_prompt(query, contexts)
        response = self.llm.predict(prompt)
        return response

In [69]:
# 数据集加载（streaming 模式 + 采样）
def load_sampled_dataset(name, config=None, split="validation", sample_size=100):
    try:
        ds = load_dataset(name, config, split=split, streaming=True)
    except Exception:
        # 如果 validation split 不存在，用 test
        ds = load_dataset(name, config, split="test", streaming=True)
    
    sampled = []
    for i, example in enumerate(ds):
        if i >= sample_size:
            break
        sampled.append(example)
    return sampled

In [70]:
# datasets = {
#     "natural_questions": load_sampled_dataset("google-research-datasets/natural_questions", split="validation", sample_size=100),
#     "trivia_qa": load_sampled_dataset("mandarjoshi/trivia_qa", "unfiltered", split="validation", sample_size=100),
#     "squad": load_sampled_dataset("rajpurkar/squad", split="validation", sample_size=100),
#     "web_questions": load_sampled_dataset("stanfordnlp/web_questions", split="test", sample_size=100),
#     "mmlu": load_sampled_dataset("cais/mmlu", "all", split="validation", sample_size=100),
#     "strategy_qa": load_sampled_dataset("wics/strategy-qa", split="validation", sample_size=100),
#     "hotpot_qa": load_sampled_dataset("hotpot_qa", split="validation", sample_size=100)  # 移除 fullwiki
# }

In [ ]:
metric_qa = load("squad")

In [72]:
import re

def clean_answer(raw: str) -> str:
    """
    去除 <think> 标签和其中的内容，并去掉多余空白
    """
    # 去掉所有 <think>…</think> 区段
    cleaned = re.sub(r"<think>.*?</think>", "", raw, flags=re.DOTALL)
    # 去掉多余的换行和空格
    return cleaned.strip()

In [80]:
ds = load_dataset("google-research-datasets/natural_questions",
                      split="validation", streaming=True)

Resolving data files:   0%|          | 0/287 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/287 [00:00<?, ?it/s]

In [83]:
def evaluate_natural_questions( 
        ds,
        clients: list[Client],
        llm_server: Server,
        sample_limit=100,
        top_k=5,
        output_csv: str = "natural_questions_results.csv"): 
    samples = []
    for i, ex in enumerate(ds):
        if i >= sample_limit:
            break
        samples.append(ex)
    print(f"Loaded {len(samples)} Natural Questions samples.")

    results = []
    for idx, sample in enumerate(samples):
        question = sample["question"]["text"]
        print(f"Processing question: {question}")
        gold_answers = sample.get("answers", {}).get("text", [])
        print(f"Gold answers: {gold_answers}")
        if not gold_answers:
            gold_answers = [""]  # 占位

        start_time = time.time()
        # 向所有client广播 query
        all_proofs = []
        for c in clients:
            proofs, q_vec = c.retrieve(question, top_k)
            all_proofs.extend(proofs)

        # 根据得分进行排序，选出最优proofs
        all_proofs.sort(key=lambda p: p.score, reverse=True)

        # 请求对应client提供真实上下文
        contexts = [r.document.page_content for r in all_proofs[:top_k]]

        # 调用 LLM Server
        answer = llm_server.generate_answer(question, contexts)
        latency = time.time() - start_time

        # 构造 SQuAD 格式输入
        prediction = [{"id": str(idx), "prediction_text": answer}]
        reference  = [{
            "id": str(idx),
            "answers": {"text": gold_answers, "answer_start": [0]}
        }]

        # 计算 EM 和 F1
        qa_metrics = metric_qa.compute(predictions=prediction, references=reference)
        em_score = qa_metrics["exact_match"]
        f1_score = qa_metrics["f1"]

        results.append({
            "question": question,
            "answer": answer,
            "em": em_score,
            "f1": f1_score,
            "latency": latency,
            "retrieved_docs": contexts
        })

        if idx % 10 == 0:
            print(f"[{idx}] EM={em_score}, F1={f1_score}, latency={latency:.2f}s")

    # 保存到 CSV
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False, encoding="utf-8-sig")
    print(f"Saved Natural Questions results to {output_csv}")

In [ ]:
clients = [Client(vectorstore_path="./common_sense_db"), Client(vectorstore_path="./computer_science_coding_related_db"),
               Client(vectorstore_path="./law_related_db"), Client(vectorstore_path="./medicine_related_db")]
for c in clients:
    c.load_vectorstore()
llm_server = Server(model_name="qwen3:4b")

llama_model_loader: loaded meta data with 36 key-value pairs and 310 tensors from ./models/Qwen3-Embedding/Qwen3-Embedding-0.6B-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen3
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen3 Embedding 0.6b
llama_model_loader: - kv   3:                           general.basename str              = qwen3-embedding
llama_model_loader: - kv   4:                         general.size_label str              = 0.6B
llama_model_loader: - kv   5:                            general.license str              = apache-2.0
llama_model_loader: - kv   6:                   general.base_model.count u32              = 1
llama_model_loader: - kv  

Vectorstore ./common_sense_db loaded.


init_tokenizer: initializing tokenizer for type 2
load: control token: 151661 '<|fim_suffix|>' is not marked as EOG
load: control token: 151649 '<|box_end|>' is not marked as EOG
load: control token: 151647 '<|object_ref_end|>' is not marked as EOG
load: control token: 151654 '<|vision_pad|>' is not marked as EOG
load: control token: 151659 '<|fim_prefix|>' is not marked as EOG
load: control token: 151648 '<|box_start|>' is not marked as EOG
load: control token: 151644 '<|im_start|>' is not marked as EOG
load: control token: 151646 '<|object_ref_start|>' is not marked as EOG
load: control token: 151650 '<|quad_start|>' is not marked as EOG
load: control token: 151651 '<|quad_end|>' is not marked as EOG
load: control token: 151652 '<|vision_start|>' is not marked as EOG
load: control token: 151653 '<|vision_end|>' is not marked as EOG
load: control token: 151655 '<|image_pad|>' is not marked as EOG
load: control token: 151656 '<|video_pad|>' is not marked as EOG
load: control token: 151

Vectorstore ./computer_science_coding_related_db loaded.


init_tokenizer: initializing tokenizer for type 2
load: control token: 151661 '<|fim_suffix|>' is not marked as EOG
load: control token: 151649 '<|box_end|>' is not marked as EOG
load: control token: 151647 '<|object_ref_end|>' is not marked as EOG
load: control token: 151654 '<|vision_pad|>' is not marked as EOG
load: control token: 151659 '<|fim_prefix|>' is not marked as EOG
load: control token: 151648 '<|box_start|>' is not marked as EOG
load: control token: 151644 '<|im_start|>' is not marked as EOG
load: control token: 151646 '<|object_ref_start|>' is not marked as EOG
load: control token: 151650 '<|quad_start|>' is not marked as EOG
load: control token: 151651 '<|quad_end|>' is not marked as EOG
load: control token: 151652 '<|vision_start|>' is not marked as EOG
load: control token: 151653 '<|vision_end|>' is not marked as EOG
load: control token: 151655 '<|image_pad|>' is not marked as EOG
load: control token: 151656 '<|video_pad|>' is not marked as EOG
load: control token: 151

Vectorstore ./law_related_db loaded.


init_tokenizer: initializing tokenizer for type 2
load: control token: 151661 '<|fim_suffix|>' is not marked as EOG
load: control token: 151649 '<|box_end|>' is not marked as EOG
load: control token: 151647 '<|object_ref_end|>' is not marked as EOG
load: control token: 151654 '<|vision_pad|>' is not marked as EOG
load: control token: 151659 '<|fim_prefix|>' is not marked as EOG
load: control token: 151648 '<|box_start|>' is not marked as EOG
load: control token: 151644 '<|im_start|>' is not marked as EOG
load: control token: 151646 '<|object_ref_start|>' is not marked as EOG
load: control token: 151650 '<|quad_start|>' is not marked as EOG
load: control token: 151651 '<|quad_end|>' is not marked as EOG
load: control token: 151652 '<|vision_start|>' is not marked as EOG
load: control token: 151653 '<|vision_end|>' is not marked as EOG
load: control token: 151655 '<|image_pad|>' is not marked as EOG
load: control token: 151656 '<|video_pad|>' is not marked as EOG
load: control token: 151

Vectorstore ./medicine_related_db loaded.


In [ ]:
evaluate_natural_questions(ds, clients, llm_server, sample_limit=100, top_k=5, output_csv="natural_questions_results.csv")

llama_perf_context_print:        load time =     101.38 ms
llama_perf_context_print: prompt eval time =      84.84 ms /    11 tokens (    7.71 ms per token,   129.66 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =      91.00 ms /    12 tokens


Loaded 100 Natural Questions samples.
Processing question: what purpose did seasonal monsoon winds have on trade
Gold answers: []
Index: 316, Distance: 495.740966796875
Document: page_content='that H is "a note of aspiration only and no letter", and that Z in Greek or Hebrew is a mere SS. The rules were not completely fixed in the 17th century. William Camden in his Remains commented, singling out some letters—Æ, K, W, and Z—not found in the classical Roman alphabet:' metadata={'source': './classified/common_sense\\Anagram.json', 'doc_id': 8, 'chunk_id': 8, 'faiss_id': 316}, Vector: [  3.074452   -15.067794     0.4433816  ...  -6.0150247  -15.306083
   0.77252626]
Index: 799, Distance: 495.740966796875
Document: page_content='that bees die after stinging; that workers remove corpses from the hive, and guard it; castes including workers and non-working drones, but "kings" rather than queens; predators including toads and bee-eaters; and the waggle dance, with the "irresistible suggestio

llama_perf_context_print:        load time =     161.33 ms
llama_perf_context_print: prompt eval time =     155.34 ms /    11 tokens (   14.12 ms per token,    70.81 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     161.70 ms /    12 tokens
llama_perf_context_print:        load time =     134.30 ms
llama_perf_context_print: prompt eval time =     128.75 ms /    11 tokens (   11.70 ms per token,    85.44 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     134.70 ms /    12 tokens


Index: 546, Distance: 284.73870849609375
Document: page_content='What really separated AMPS from older systems is the "back end" call setup functionality. In AMPS, the cell centers could flexibly assign channels to handsets based on signal strength, allowing the same frequency to be re-used, without interference, if locations were separated enough. The channels were grouped so a specific set was different of the one used on the cell nearby. This allowed a larger number of phones to be supported over a geographical area. AMPS pioneers coined the term "cellular" because of its use of small hexagonal "cells" within a system.' metadata={'source': './classified/computer_science_coding_related\\Advanced Mobile Phone System.json', 'doc_id': 14, 'chunk_id': 7, 'faiss_id': 546}, Vector: [  3.9952266  -16.864693     0.44079113 ...  -6.176448   -14.745873
   0.47113892]
Index: 38, Distance: 495.740966796875
Document: page_content='that code does not have to depend on the specific details of suppo

llama_perf_context_print:        load time =     190.28 ms
llama_perf_context_print: prompt eval time =     184.15 ms /    11 tokens (   16.74 ms per token,    59.73 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     190.54 ms /    12 tokens


...

Index: 2016, Distance: 109.63043975830078
Document: page_content='power, as well as interest and information is so small.  Larger institutional investors also grant the board proxies. The large number of shareholders also makes it hard for them to organize. However, there have been moves recently to try to increase shareholder activism among both institutional investors and individuals with small shareholdings.' metadata={'source': './classified/law_related\\Board of directors.json', 'doc_id': 49, 'chunk_id': 22, 'faiss_id': 2016}, Vector: [  3.5013971  -15.122153     0.29181924 ...  -5.1982903  -13.840278
   0.44439858]
Index: 2205, Distance: 109.63043975830078
Document: page_content='power" specified by Article III of the U.S. Constitution. Justice Oliver Wendell Holmes Jr. summarized centuries of history in 1917, "judges do and must legislate." In the United States, state courts continue to exercise full common law powers, and create both general common law and interstitial common l

llama_perf_context_print:        load time =     101.38 ms
llama_perf_context_print: prompt eval time =      62.88 ms /    10 tokens (    6.29 ms per token,   159.02 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =      65.37 ms /    11 tokens
llama_perf_context_print:        load time =     161.33 ms
llama_perf_context_print: prompt eval time =      88.96 ms /    10 tokens (    8.90 ms per token,   112.41 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =      93.08 ms /    11 tokens


Processing question: how many words are in national anthem of pakistan
Gold answers: []
Index: 131, Distance: 627.488037109375
Document: page_content='Another significant trend regarding future of advertising is the growing importance of the niche market using niche or targeted ads. Also brought about by the Internet and the theory of the long tail, advertisers will have an increasing ability to reach specific audiences. In the past, the most efficient way to deliver a message was to blanket the largest mass market audience possible. However, usage tracking, customer profiles and the growing popularity of niche content brought about by everything from blogs to social networking sites, provide advertisers with audiences that are smaller but much better defined, leading to ads that are more relevant to viewers and more effective for companies' marketing products. Among others, Comcast Spotlight is one such advertiser employing this method' metadata={'source': './classified/common_sense\\

llama_perf_context_print:        load time =     134.30 ms
llama_perf_context_print: prompt eval time =      85.19 ms /    10 tokens (    8.52 ms per token,   117.38 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =      89.05 ms /    11 tokens
llama_perf_context_print:        load time =     190.28 ms
llama_perf_context_print: prompt eval time =      71.86 ms /    10 tokens (    7.19 ms per token,   139.17 tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =      80.62 ms /    11 tokens


Index: 2984, Distance: 280.7076416015625
Document: page_content='How, an animal being pregnant, its young may be consecrated while still unborn (§§ 1-3). The form of words with which a temurah is made.
 Things that may not be placed on the altar (§§ 1-4). The young of animals which may not be placed on the altar may be sacrificed; sacrificial animals which have become unfit (terefah) through sickness may not be redeemed (§ 5).
 In what ways things which have been consecrated for the altar are different from things which are dedicated only for the maintenance of the Temple, and in what ways they are similar (§§ 1-3). What sacrificial objects must be burned and what buried; in this connection are enumerated other unconsecrated things which must be partly burned and partly buried (§§ 4-6).' metadata={'source': './classified/law_related\\Temurah (Talmud).json', 'doc_id': 98, 'chunk_id': 3, 'faiss_id': 2984}, Vector: [  4.16083    -16.211777     0.35981044 ...  -5.812963   -12.445858
   0.5